### **Цуприков Дмитрий, ИУ5-63Б, РК2 ТМО, вариант 21**

**Задание**. Для заданного набора данных (https://www.kaggle.com/datasets/oreojam/formula-e-world-championship-race-results) постройте модели классификации или регрессии (буду строить модели регрессии). Для построения моделей используйте методы 1 и 2 (для ИУ5-63Б это - дерево решений и случайный лес). Оцените качество моделей на основе подходящих метрик качества (соответственно, MSE, MAE, R^2). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

Произведем импорт необходимых библиотек и их элементов. Создадим датафрейм race в библиотеĸе Pandas на основе имеющегося csv-файла:

In [191]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import re

In [192]:
race = pd.read_csv("Formual_E_Raceresults.csv")
race

,SeasonName,RaceName,Pos,DriverNumber,DriverFirstName,DriverLastName,Team,Started,Best,Time,PtsPoints
0,Season 1 2014/15,Beijing E-Prix,1,#11,Lucas,Di Grassi,Audi Sport ABT Formula E Team,2,1:46.718,52:23.413,25
1,Season 1 2014/15,Beijing E-Prix,2,#27,Franck,Montagny,Andretti Autosport Formula E Team,8,1:46.640,+ 2.867,18
2,Season 1 2014/15,Beijing E-Prix,3,#2,Sam,Bird,Virgin Racing Formula E Team,11,1:46.563,+ 6.559,15
3,Season 1 2014/15,Beijing E-Prix,4,#28,Charles,Pic,Andretti Autosport Formula E Team,7,1:46.730,+ 19.301,12
4,Season 1 2014/15,Beijing E-Prix,5,#5,Karun,Chandhok,Mahindra Racing Formula E Team,4,1:45.892,+ 23.952,10
...,...,...,...,...,...,...,...,...,...,...,...
1497,Season 7 2020/21,Monaco E-Prix - R7,0,#17,Nyck,De Vries,Mercedes-EQ Formula E Team,24,1:36.102,–DNF,0
1498,Season 7 2020/21,Monaco E-Prix - R7,0,#99,Pascal,Wehrlein,TAG Heuer Porsche Formula E Team,8,1:35.717,–DNF,0
1499,Season 7 2020/21,Monaco E-Prix - R7,0,#5,Stoffel,Vandoorne,Mercedes-EQ Formula E Team,15,1:34.428,–DNF,0
1500,Season 7 2020/21,Monaco E-Prix - R7,0,#33,René,Rast,Audi Sport ABT Schaeffler,10,1:34.586,–DNF,0


In [193]:
race.shape

(1502, 11)

In [194]:
race.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SeasonName       1502 non-null   object
 1   RaceName         1502 non-null   object
 2   Pos              1502 non-null   int64 
 3   DriverNumber     1502 non-null   object
 4   DriverFirstName  1502 non-null   object
 5   DriverLastName   1502 non-null   object
 6   Team             1502 non-null   object
 7   Started          1502 non-null   object
 8   Best             1502 non-null   object
 9   Time             1502 non-null   object
 10  PtsPoints        1502 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 129.2+ KB


Проверим исходный набор данных на наличие пропусков, их там не оказалось

In [195]:
race.isnull().sum()

SeasonName         0
RaceName           0
Pos                0
DriverNumber       0
DriverFirstName    0
DriverLastName     0
Team               0
Started            0
Best               0
Time               0
PtsPoints          0
dtype: int64

Проверим исходный набор данных на наличие записей-дубликатов, их там не оказалось

In [196]:
race.duplicated().sum()

0

Для сокращения времени построения моделей можно использовать фрагмент набора данных (например, первые 200-500 строк). Поэтому преобразуем исходный датафрейм размером в 1502 строки к 200 строкам.

In [197]:
race = race.iloc[:200]

In [198]:
race.shape

(200, 11)

Во избежание перенасыщения датафрейма излишними колонками при кодировании категориальных значений, трансформируем сами некоторые из них (три столбца: Started, DriverNumber, Best) из типа object в числовые типы.

Функция для удаления из поля  колонки всех нецифровых символов, используем при преобразовании значений столбца Started.

In [199]:
def clean_and_convert(value):
    if isinstance(value, str):
        cleaned_value = re.sub(r'\D', '', value)
        return int(cleaned_value)
    return value

Функция для предназначена для обработки значений временных данных, представленных в формате строки, и приведению их к вещественному виду;
используем при преобразовании значений столбца Best.

In [200]:
def process_best_time(value):
    if value == '-':
        return 0.0
    if isinstance(value, str) and value.strip():
        value = re.sub(r'[^\d:.]', '', value)
        if ':' in value:
            minutes, seconds = value.split(':')
            total_seconds = int(minutes) * 60 + float(seconds)
            return total_seconds
    return 0.0

Обрабатываем данные в столбцах, а затем преобразуем их к числовому типу с помощью функции to_numeric.

In [201]:
race.loc[:, 'DriverNumber'] = race['DriverNumber'].astype(str).apply(lambda x: int(re.sub(r'\D', '', x)))

race.loc[:, 'Best'] = race['Best'].apply(process_best_time).astype(float)

race.loc[:, 'Started'] = race['Started'].apply(clean_and_convert)

race[['DriverNumber', 'Best', 'Started']] = race[['DriverNumber', 'Best', 'Started']].apply(pd.to_numeric)

<ipython-input-201-e21d12d3b1a2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race[['DriverNumber', 'Best', 'Started']] = race[['DriverNumber', 'Best', 'Started']].apply(pd.to_numeric)


In [202]:
race.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SeasonName       200 non-null    object 
 1   RaceName         200 non-null    object 
 2   Pos              200 non-null    int64  
 3   DriverNumber     200 non-null    int64  
 4   DriverFirstName  200 non-null    object 
 5   DriverLastName   200 non-null    object 
 6   Team             200 non-null    object 
 7   Started          200 non-null    int64  
 8   Best             200 non-null    float64
 9   Time             200 non-null    object 
 10  PtsPoints        200 non-null    int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 17.3+ KB


Применяем one-hot кодирование для всех колонок, содержащих категориальные данные, за исключением столбца Time, так как мы не будем использовать его при построении моделей.

In [203]:
columns_to_encode = [col for col in race.columns if col != 'Time' and race[col].dtype == 'object']
race = pd.get_dummies(race, columns=columns_to_encode)

In [204]:
print(race.columns)

Index(['Pos', 'DriverNumber', 'Started', 'Best', 'Time', 'PtsPoints',
       'SeasonName_Season 1 2014/15', 'RaceName_Beijing E-Prix',
       'RaceName_Berlin E-Prix', 'RaceName_Buenos Aires ePrix',
       'RaceName_London E-Prix', 'RaceName_Long Beach E-Prix',
       'RaceName_Miami E-Prix', 'RaceName_Monaco E-Prix',
       'RaceName_Moscow E-Prix', 'RaceName_Punta del Este E-Prix',
       'RaceName_Putrajaya E-Prix', 'DriverFirstName_Alex',
       'DriverFirstName_Antonio', 'DriverFirstName_António Félix',
       'DriverFirstName_Bruno', 'DriverFirstName_Charles',
       'DriverFirstName_Daniel', 'DriverFirstName_Fabio',
       'DriverFirstName_Franck', 'DriverFirstName_Ho-Pin',
       'DriverFirstName_Jaime', 'DriverFirstName_Jarno',
       'DriverFirstName_Jean-Éric', 'DriverFirstName_Justin',
       'DriverFirstName_Jérôme', 'DriverFirstName_Karun',
       'DriverFirstName_Katherine', 'DriverFirstName_Loïc',
       'DriverFirstName_Lucas', 'DriverFirstName_Marco',
       'DriverFi

В качестве целевого признака возьмем содержимое столбца PtsPoints, то есть задачей регрессии будет являться прогнозирование количества очков, заработанных гонщиком, которые определяют его финальное место. Помимо Time также при построении модели не учитываем признак SeasonName_Season 1 2014/15, так как первые 200 строк текущего датафрейма, отсеченные в начале от исходного, характеризуют один и тот же сезон, поэтому значение этой колонки будет неизменно True.

In [207]:
target_column = 'PtsPoints'
X = race.drop([target_column,'Time', 'SeasonName_Season 1 2014/15'], axis=1)
y = race[target_column]

In [208]:
# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Модель Decision Tree Regressor
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

# Модель Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [209]:
mse_dt = mean_squared_error(y_test, y_pred_dt)
mae_dt = mean_absolute_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Вывод результатов
print("Decision Tree Regressor:")
print(f"MSE: {mse_dt}")
print(f"MAE: {mae_dt}")
print(f"R^2: {r2_dt}")

print("\nRandom Forest Regressor:")
print(f"MSE: {mse_rf}")
print(f"MAE: {mae_rf}")
print(f"R^2: {r2_rf}")

Decision Tree Regressor:
MSE: 0.325
MAE: 0.175
R^2: 0.9898278560250391

Random Forest Regressor:
MSE: 0.7314975
MAE: 0.38824999999999993
R^2: 0.9771049295774648


В этом случае были использованы три распространенные метрики качества регрессионных моделей:

**MSE (Mean Squared Error)**: Это средняя квадратичная ошибка, которая измеряет среднеквадратичное отклонение прогнозируемых значений от истинных значений. Она показывает, насколько сильно среднеквадратичное отклонение наших прогнозов от реальных значений. Чем меньше значение MSE, тем лучше.

**MAE (Mean Absolute Error)**: Это средняя абсолютная ошибка, которая измеряет среднее абсолютное отклонение прогнозируемых значений от истинных значений. Она показывает среднее абсолютное отклонение наших прогнозов от реальных значений. Как и MSE, ниже значение MAE, тем лучше.

**R^2 (R-squared)**: Коэффициент детерминации, который измеряет долю дисперсии зависимой переменной, которая объясняется моделью. Он показывает, насколько хорошо модель соответствует данным. Значение R^2 близкое к 1 означает хорошее соответствие модели данным.

Использование этих метрик позволяет оценить качество построенных моделей:

**Decision Tree Regressor**: У данной модели низкая среднеквадратичная и средняя абсолютная ошибка, а также высокий коэффициент детерминации, что свидетельствует о хорошем качестве аппроксимации моделью исходных данных.

**Random Forest Regressor**: Среднеквадратичная и средняя абсолютная ошибка у этой модели выше, а коэффициент детерминации ниже по сравнению с решающим деревом. Возможно, это связано с тем, что случайный лес более сложная модель, и требуется тщательная настройка гиперпараметров для достижения лучшего качества. Возможно, текущая конфигурация случайного леса не оптимальна для данного набора данных.